In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from keras.layers import GlobalAveragePooling2D,Flatten,Dense,Dropout
from keras import layers
!pip install --quiet /kaggle/input/efficientnet-git
import efficientnet.keras as efn



#from tensorflow.keras.applications import EfficientNetB0

from tensorflow.keras.applications import EfficientNetB3

import cv2

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_path='../input/cassava-leaf-disease-classification/train_images'
df=pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
df.head()
new_df1=df
new_df=df
df.shape

new_df['weights']=np.where(new_df['label']==3,1,0)
new_df=new_df.sample(n=2500,weights=new_df.weights,random_state=42)
sampled_df=new_df.drop(['weights'],axis=1)
sampled_df.count()


index=new_df1[new_df1['label']==3].index

new_df1.drop(index,inplace=True)

new_df1['label'].value_counts()

#new_df1.append(new_df1,ignore_index=True)

final_df=new_df1.append(sampled_df,ignore_index=True)
final_df['label'].value_counts()

In [ ]:
train,val=train_test_split(df,test_size=0.05,random_state=123)
print(train.shape)
print(val.shape)
sns.countplot(df['label'])

In [ ]:
sns.countplot(df['label'])

shapes=[]
df_train_id=df['image_id']
range(len(df_train_id))

for i in range(len(df_train_id)):
    img_id=df_train_id[i]
    img_path = os.path.join(train_path, img_id)
    shapes.append(cv2.imread(img_path).shape)
    


shapes_df=pd.DataFrame({'Shapes':shapes})
shapes_df.to_csv('imageSizes.csv')
shapes_df.value_counts()

In [ ]:
train['label']=train['label'].astype('string')
val['label']=val['label'].astype('string')

#converting to string because when class_mode=categorical we need strings or lists

# using image data generator to read files 

In [ ]:
print(train.shape)
print(val.shape)

In [ ]:
train_generator = ImageDataGenerator(preprocessing_function = tf.keras.applications.efficientnet.preprocess_input,
                                     rotation_range = 0,
                                     width_shift_range = 0.2,
                                     height_shift_range = 0.2,
                                     shear_range = 0.5,
                                     zoom_range = 0.2,
                                     horizontal_flip = True,
                                     vertical_flip = True,
                                     fill_mode = 'reflect').flow_from_dataframe(
                            train,
                            directory = train_path,
                            x_col = "image_id",
                            y_col = "label",
                            #weight_col = None,
                            target_size = (256, 256),
                            #color_mode = "rgb",
                            #classes = None,
                            class_mode = "categorical",
                            batch_size = 32,
                            shuffle = True,
                            #seed = 34,
                            #save_to_dir = None,
                            #save_prefix = "",
                            #save_format = "png",
                            
                            interpolation = "nearest",
                            #validate_filenames = True
)


In [ ]:
batch=next(train_generator)
print(batch[0].shape)
image=(batch[0][2])/255
plt.imshow(image)

In [ ]:
valid_generator = ImageDataGenerator(preprocessing_function = tf.keras.applications.efficientnet.preprocess_input,).flow_from_dataframe(
                            val,
                            directory = train_path,
                            x_col = "image_id",
                            y_col = "label",
                            #weight_col = None,
                            target_size = (256, 256),
                            #color_mode = "rgb",
                            #classes = None,
                            class_mode = "categorical",
                            batch_size = 32,
                            shuffle = True,
                            #seed = 34,
                            #save_to_dir = None,
                            #save_prefix = "",
                            #save_format = "png",
                            
                            interpolation = "nearest",
                            #validate_filenames = True
)

In [ ]:
batch1=next(valid_generator)

In [ ]:
print(batch1[0].shape)
plt.imshow((batch1[0][2])/255)

# building the efficient net b3 model

In [ ]:
def modelEfficientNetB3():
    model=keras.Sequential()
    model.add(EfficientNetB3(include_top=False,
                             weights='imagenet',
                             input_shape=(256,256,3)
                            
                            ))
    model.add(GlobalAveragePooling2D()),
    model.add(Flatten())
    
    model.add(Dense(256, activation = 'relu', bias_regularizer=tf.keras.regularizers.L1L2(l1=0.01, l2=0.001)))
    model.add(Dropout(0.5))
    model.add(Dense(5,activation='softmax'))
    
    return model
    

In [ ]:
model=modelEfficientNetB3()

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras import utils

In [ ]:
utils.plot_model(model)

In [ ]:
opt=keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics='accuracy')

In [ ]:
history=model.fit_generator(train_generator,epochs=25,validation_data=valid_generator)

In [ ]:
model.save('./EffNetB4_300_try8.h5')


In [ ]:
history_df_save8=pd.DataFrame(history.history)
history_df_save8.to_csv('history_undersampling8.csv',index=False)

# Predictions

In [ ]:
ss = pd.read_csv("../input/cassava-leaf-disease-classification/sample_submission.csv")
preds = []

for image_id in ss.image_id:
    image = cv2.cvtColor(cv2.imread('../input/cassava-leaf-disease-classification/test_images/'+image_id),cv2.COLOR_BGR2RGB)
    image = cv2.resize(image,(224,224))
    image = np.expand_dims(image, axis = 0)
    preds.append(np.argmax(model.predict_generator(image)))

ss['label'] = preds
ss.to_csv('submission.csv', index = False)

In [ ]:
sample=pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
sample.head()

In [ ]:



ss.head()